## Importing Necessary Libraries

In [11]:
import warnings
warnings.filterwarnings('ignore')
import os, evaluate, torch, accelerate, nltk, re, datetime 
import pandas as pd
from collections import Counter
import numpy as np
from datasets import load_dataset, Dataset, DatasetDict
from transformers import BertTokenizer
from transformers import AutoTokenizer, AutoModel
from transformers import AutoModelForTokenClassification
from datasets import load_metric
from transformers import TrainingArguments, Trainer
from transformers import pipeline
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn import metrics
import seaborn as sns
from pynvml import *
from numba import cuda
import matplotlib.pyplot as plt

## Get Current Working Directory

In [12]:
my_dir = os.getcwd() + '/' 
print(f"CWD: {my_dir}")

CWD: /home/sharjeelahmed/datascience/


## GPU Configuration

<p> Reset Cuda Memory</p>

In [13]:
device = cuda.get_current_device()
device.reset()
with torch.no_grad():
    torch.cuda.empty_cache()

<p>Check GPU Memory</p>

In [14]:
nvmlInit()
h = nvmlDeviceGetHandleByIndex(0)
info = nvmlDeviceGetMemoryInfo(h)
print(f'Total    : {info.total//1024**2} MB')
print(f'Free     : {info.free//1024**2} MB')
print(f'Used     : {info.used//1024**2} MB')

Total    : 15360 MB
Free     : 14490 MB
Used     : 869 MB


### Get Cuda Device

In [15]:
devices = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
devices

device(type='cuda', index=0)

## Load Modela nd Tokenizer

In [16]:
tokenizer = AutoTokenizer.from_pretrained("Clinical-AI-Apollo/Medical-NER")
pre_trained_model = AutoModelForTokenClassification.from_pretrained("Clinical-AI-Apollo/Medical-NER")

In [17]:
pipe = pipeline("token-classification", tokenizer = tokenizer , model = pre_trained_model)

In [18]:
hpi = """
Patient was referred to Physical Therapy due to change in functional performance of: walking,
transfers, bed mobility, and wheelchair mobility
When: Approximately 02-19-21
Factors contributing to functional change: recent hospitalization, change in previous condition, and change in balance.
Additional comments including complication, comorbidity, or safety issue warranting skilled care: Pt admitted to hospital 3
weeks ago w/ a kidney infection secondary to a kidney stone. He presents w/ signficant deconditioning and gait disturbance,
which limit his ability to safely transfer and ambulate within his SLC.
Recomendation should be Panadol, Brufene, Paracetamol and one tble spoon of pink salt.
"""

In [20]:
pred = pipe(hpi)

In [21]:
print("Index | Word     | Entity                     | Score")
print("-" * 75)
for item in pred:
    print(f"{item['index']:5} | {item['word']:8} | {item['entity']:20} | {item['score']:.5f}")


Index | Word     | Entity                     | Score
---------------------------------------------------------------------------
    3 | ▁referred | B-CLINICAL_EVENT     | 0.89074
    5 | ▁Physical | B-NONBIOLOGICAL_LOCATION | 0.64057
    6 | ▁Therapy | I-NONBIOLOGICAL_LOCATION | 0.65925
   15 | ▁walking | B-DIAGNOSTIC_PROCEDURE | 0.68079
   17 | ▁transfers | B-DIAGNOSTIC_PROCEDURE | 0.67863
   19 | ▁bed     | B-BIOLOGICAL_STRUCTURE | 0.25953
   20 | ▁mobility | I-DIAGNOSTIC_PROCEDURE | 0.32647
   23 | ▁wheelchair | B-BIOLOGICAL_STRUCTURE | 0.21599
   24 | ▁mobility | I-DIAGNOSTIC_PROCEDURE | 0.25045
   28 | ▁02      | B-DATE               | 0.46283
   29 | -        | I-DATE               | 0.78039
   30 | 19       | I-DATE               | 0.75045
   31 | -        | I-DATE               | 0.71389
   32 | 21       | I-DATE               | 0.68596
   68 | ▁admitted | B-CLINICAL_EVENT     | 0.85494
   70 | ▁hospital | B-NONBIOLOGICAL_LOCATION | 0.77419
   71 | ▁3       | B-DATE          